#  Scotch whisky selection design

The grocery store would like to choose a selection of whiskies to offer so as to maximize the expected per-customer revenue.


## Part 1: Solving the problem
a) Analyzing the expected revenue of the optimal selection of whiskies.

b) Which whiskies are offered in the optimal solution.

c) The choice probabilities of the whiskies in the optimal solution.

In [210]:
import numpy as np
import pandas as pd

# Read the partworth matrix:
price = pd.read_csv("whiskies-attributes-prices.csv")
price = np.asarray(price)

uti = pd.read_csv("whiskies-utilities.csv")
uti = np.asarray(uti)

marginal_profit = np.transpose(price)[13]
marginal_profit

array([45, 86, 52, 60, 105, 80, 140, 125, 60, 85, 136, 140, 65, 58, 150,
       63, 57, 85, 90, 66, 65, 75, 58, 90, 65, 70, 80, 63, 69, 61, 225,
       68, 115, 185, 65, 49, 165, 36, 165, 85, 200, 146, 64, 79, 115, 63,
       50, 70, 90, 175, 86, 45, 220, 54, 90, 77, 190, 115, 130, 100, 79,
       110, 72, 180, 100, 110, 83, 105, 56, 73, 500, 86, 60, 58, 67, 199,
       160, 73, 67, 165, 110, 105, 35, 90, 110, 78], dtype=object)

In [119]:
utility = uti.copy()
S = np.array([1,2,3,4,5])
#S = np.arange(1,101,1)
nCustomers = 100
nProducts = 86
#utility = np.transpose(utility)

# expected profit function
def expected_profit(S):
    # Add the no-purchase option:
    # NB: the products are numbered from 0 to 99. Index 100 ( = nProducts) will correspond to the no-purchase option. 
    S2 = np.append(S, nProducts-1)
    
    choice_probabilities = np.zeros([nCustomers,len(S2)])
    #for i in S2:
        #choice_probabilities[i] = 0.0;
    
    for k in range(nCustomers):
        #utilities
        uti = utility[k][S-1]
        # first choice = max utilities
        first_choice = np.argmax(uti)
        # Compare to non_purchase uti 27
        if (uti[first_choice] < 27):
            first_choice = len(S)
        choice_probabilities[k][first_choice] = 1
        

        #ind = np.argmax( utilities_mat[k,S2])
#         if (S2[ind] == 100):
#             print utilities_mat[k,S2]         
        #choice_probabilities[ S2[ind] ] += 1.0/nCustomers
    
    marginal_p = np.append(marginal_profit[S-1],0)
    exp_revenure = np.dot(choice_probabilities, marginal_p)
    mean_exp_revenue = exp_revenure.mean()
    #exp_revenue = sum( [choice_probabilities[i] * marginal_profit[i] for i in S])
        
    return mean_exp_revenue, choice_probabilities


# Let's see one simple example -- we only offer product 1.
exp_profit, choice_probabilities = expected_profit(S)
print("Expected profit: ", exp_profit)
#print("Choice probabilities: ", choice_probabilities)

('Expected profit: ', 56.31)


In [108]:
all_product = np.arange(1,nProducts+1,1)
exp_profit, choice_probabilities = expected_profit(all_product)
print("Expected profit: ", exp_profit)
print("Choice probabilities: ", choice_probabilities)


('Expected profit: ', 106.2)
('Choice probabilities: ', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))


In [129]:
len(marginal_profit)
range(nProducts)
marginal_profit[85]

78

In [123]:
utility.shape

nopurchase_utilities = np.full((nCustomers,1),27)

utility2 = np.concatenate( (utility, nopurchase_utilities) , axis = 1)
utility2.shape

(100, 87)

In [372]:
from gurobipy import *

m = Model()

# Create the decision variables
x = m.addVars(nProducts, vtype = GRB.BINARY )
y = m.addVars(nCustomers, nProducts+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints:")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utility2[k,j] * y[k,j] for j in range(nProducts+1)) >= utility2[k,i] * x[i] + utility2[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( utility2[k,j] * y[k,j] for j in range(nProducts+1)) >= utility2[k,nProducts] )



# Create the objective:
print("Creating objective:")
m.setObjective( quicksum(marginal_profit[i] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()


S_optimal = [i for i in range(nProducts) if x[i].x == 1]
print("Optimal set of products: ", S_optimal)

optimal_profit = m.objval
print("Optimal profit: ",optimal_profit)

# Verify that optimal profit is correct:
exp_profit_S_optimal, choice_probabilities_S_optimal = expected_profit( all_product )
print("Optimal profit (via expected_profit): ", exp_profit_S_optimal)

Creating constraints:
Creating objective:
Update completed
Optimize a model with 17400 rows, 8786 columns and 339435 nonzeros
Variable types: 0 continuous, 8786 integer (8786 binary)
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [3e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 12495 rows and 6281 columns
Presolve time: 0.63s
Presolved: 4905 rows, 2505 columns, 55538 nonzeros
Variable types: 0 continuous, 2505 integer (2505 binary)
Found heuristic solution: objective 107.5200000

Root relaxation: objective 1.345310e+02, 605 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  134.53104    0  340  107.52000  134.53104  25.1%     -    0s
H    0     0                     121.3700000  134.53104  10.8%     -    0s
H    0     0                     128.2800000  134.53104  4.87%

In [345]:
S_optimal2=np.array(S_optimal)
s,cp=expected_profit(S_optimal2+1)
# print(y)
# choice probabilities
a=[sum(x) for x in zip(*cp)]
whiskies_cp = np.array(np.array(a)/100)
print(whiskies_cp)
np.array(whiskies_cp)[8]
print(s)
sum(whiskies_cp)

[0.07 0.06 0.05 0.05 0.08 0.07 0.   0.24 0.07 0.14 0.07 0.02 0.08]
130.35


0.9999999999999999

In [346]:
# name of optimal set
print(price[:,0][S_optimal2])
print("Product & Python index ",S_optimal2)

['Auchentoshan' 'Balblair' 'Balmenach' 'Balvenie' 'Bowmore'
 'GlenDeveronMacduff' 'Glenallachie' 'Lagavulin' 'Laphroiag' 'Miltonduff'
 'RoyalLochnagar' 'Tomatin']
('Product & Python index ', array([ 6,  9, 10, 11, 18, 32, 41, 57, 58, 64, 70, 82]))


## Part 2: Balancing the whisky selection

In [363]:
# Laphroaig
a= 58
print(price[a])
#print(y[4,56].x)
b=a
sum(y[k,b].x for k in range(nCustomers))/100

['Laphroiag' 4 2 4 4 1 0 0 1 1 1 0 0 130]


0.07

## Part 3: Extending the formulation

### The grocery store is interested in enriching the formulation by incorporating additional requirements on the selection of whiskies. 

a) The fraction of customers choosing Laphroaig must be at least 0.04.

In [373]:
m.addConstr( sum(y[k,58] for k in range(nCustomers))/100 >= 0.04) # no more than 5 products

m.update()
m.optimize()

# Retrieve the optimal solution and objective value:
S_optimal_constrained = [i for i in range(nProducts) if x[i].x == 1]
print("Optimal set of products (width of product line at most 5): ", S_optimal_constrained)

optimal_profit_constrained = m.objval
print("Optimal profit (width of product line at most 5): ", optimal_profit_constrained)

Optimize a model with 17401 rows, 8786 columns and 339535 nonzeros
Variable types: 0 continuous, 8786 integer (8786 binary)
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [3e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-02, 3e+01]

Loaded MIP start with objective 130.35

Presolve removed 13186 rows and 6607 columns
Presolve time: 0.83s
Presolved: 4215 rows, 2179 columns, 49755 nonzeros
Variable types: 0 continuous, 2179 integer (2179 binary)

Root relaxation: objective 1.322957e+02, 522 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  132.05614    0  240  130.35000  132.05614  1.31%     -    1s
     0     0  130.92074    0  272  130.35000  130.92074  0.44%     -    1s
     0     0  130.90629    0  269  130.35000  130.90629  0.43%     -    1s
     0     0  130.90158    0  268  130.35000  130.901

In [380]:
S_optimal3=np.array(S_optimal_constrained)
s,cp=expected_profit(S_optimal3+1)
# print(y)
# choice probabilities
a=[sum(x) for x in zip(*cp)]
np.array(np.array(a)/100)

array([0.07, 0.06, 0.05, 0.05, 0.08, 0.07, 0.  , 0.24, 0.07, 0.14, 0.07,
       0.02, 0.08])

In [388]:
print(price[18][0],price[66][0],price[77][0])

('Bowmore', 'Oban', 'Talisker')
